# Python中的定点小数(`Decimal`)和分数(`Fraction`)对象, 以及支持的运算

In [1]:
import math, decimal as deci, fractions as frac; 
from decimal import Decimal; from fractions import Fraction; 

## 定点小数的构造及其精度管理

### 浮点数的局限性
若采用浮点数表示一个有理数, 当有理数的最简分数的分母质因数包括$2_{(10)}$以外的质数时, 由于被表示的数在二进制下是无限循环小数, 因此采用浮点数保留其在二进制下的53位有效数字, 将截断后续的部分, 从而产生舍入误差. 

例如: 十进制下的两个有限小数$0.25_{(10)}$和$0.2_{(10)}$, 在二进制下将分别表示为

|十进制小数|十进制分数|二进制小数|
|:-|:-:|:-|
|$0.25_{(10)}$ (有限小数)|$1\over4$|$0.01_{2}$ (有限小数)|
|$0.2_{(10)}$ (有限小数)|$1\over5$|$0.\dot{0}01\dot{1}_{(2)}$ (无限循环小数)|

In [2]:
xFlt = 0.25; xFlt2Dec = Decimal(xFlt); xStr2Dec = Decimal("0.25"); 
print([x.as_integer_ratio() for x in [xFlt, xFlt2Dec, xStr2Dec]]); 
print(xFlt == xFlt2Dec, xFlt2Dec == xStr2Dec)

[(1, 4), (1, 4), (1, 4)]
True True


### 基于不同方法的`Decimal`定义及其机制
上述二进制下的无限循环小数, 分别用不同的方法定义, 定义机制各有差异: 
* 如果声明为`float`对象, 其在Python中的实际值与字面值将存在微小差异. 
> 例如, `yFlt`是通过直接在代码中输入十进制小数
> ```Python
> 0.2 #小数的两端没有任何形式的括注
> ```
> 定义得到的浮点数, 实际的表示结果是
> ```Python
> 0.200000000000000011102230246251565404236316680908203125
> ```
> 同时, 浮点数与浮点数计算, 过程中不仅会将这一误差放大, 还会引入新的误差并累计. 

* 如果用浮点数直接声明`Decimal`对象, 则会将浮点数按照**实际的表示结果**进行转换, 实际值与转换前的`float`一致, 仍与字面值存在微小差异. 
> 例如, `yFlt2Dec`是将(被解释器视为浮点数的)十进制小数直接用`Decimal`函数转换, 所构造的定点数对象. 
> ```Python
> Decimal(0.2) #把小数直接用Decimal(∙)括注
> ```
> 实际的表示结果也是
> ```Python
> 0.200000000000000011102230246251565404236316680908203125
> ```
> 此时浮点数被转化为十进制定点数, 与其他定点数的计算过程中, 原有的误差会被放大, 但不会引入新的误差. 

* 当它写作**被字符串引号括注后的**十进制小数的形式, 并被声明为`Decimal`对象, 则其在Python中的实际值将与字面值一致. 
> 例如, `yStr2Dec`是将十进制小数写作**被双引号括注后的字符串形式**后, 进一步, 通过
> ```Python
> Decimal("0.2") #先把小数用双引号括注, 再用Decimal(∙)括注
> ```
> 被`Decimal`函数声明, 所构造的定点数对象. 


In [3]:
yFlt = 0.2; yFlt2Dec = Decimal(0.2); yStr2Dec = Decimal("0.2"); 
print([y.as_integer_ratio() for y in [yFlt, yFlt2Dec, yStr2Dec]]); 

[(3602879701896397, 18014398509481984), (3602879701896397, 18014398509481984), (1, 5)]


### 基于不同定义方法所得`Decimal`对象的舍入误差特征
由上一计算单元的输出结果可见: 
* `yFlt`(即`0.2`)和`yFlt2Dec`(即`Decimal(0.2)`)是相等的, 但都不等于数学意义上的0.2; 其中

    * `yFlt`是`float`对象, 在构造时即产生了舍入误差, 在后续计算中逐级传递, 并**与计算中新引入的误差结合**; 
    
    * `yFlt2Dec`(即`Decimal(0.2`)是**由`float`转换所得的**`Decimal`对象, 在构造时, 保留了浮点数`0.2`存在的舍入误差, 并且在将浮点转化为定点数的过程中, 将这个误差忠实地记录下来; \
        转化为定点数之后, 后续计算中不会引入新的误差, 只有**转换前的误差会被传递**; 
        
* 只有`yStr2Dec`(即`Decimal("0.2"`)是**由`str`构造所得的**`Decimal`对象, 其结果等于数学意义上的0.2, 在四则运算过程中产生或传递舍入误差的情形, 较`float`更少. 这是因为: 
> `Decimal`对象以十进制形式存储小数, 且对象在内存中的大小可以变化, 因此它可以表示不同精度的小数. 

由于以上特性, `Decimal`可被应用于金融计算领域. 以减少在频繁将不同数量级的数额进行累加, 或将相近的数额相减以计算差额的过程中, 采用`float`计算所引入和放大的舍入误差. 
> 在上述计算中, 若采用浮点数, 会发生"大数目'吃'小数目"现象. 因为不同数量级的浮点数, 在相加过程中, 需进行"对阶"操作. 
>> 将较小数目的小数点左移, 使得指数与最大的数目一致. \
>> 将两个以科学计数法表示的数, 徒手相加的过程中, 同样存在与这一"对阶"操作相同的环节. 
>
> 若不同数目间数量级差异悬殊, 数量级较小的数目, 将在对阶后丢失更多的有效数字. 

In [4]:
print(yFlt == yFlt2Dec, yFlt2Dec == yStr2Dec)
print([y * 39 / 13 for y in [yFlt, yFlt2Dec, yStr2Dec]])

True False
[0.6000000000000001, Decimal('0.6000000000000000333066907388'), Decimal('0.6')]


### `Decimal`的精度与修约管理
以小数的字符串形式构造的`Decimal`对象, 还将指定十进制小数的有效数字的数目. 
> 不同有效数字的`Decimal`对象, 在部分初等运算(包括四则运算, `Decimal`对象的`exp`, `ln`, `log10`等方法)所得的结果, 将采取"四舍六入五留双"的修约措施, 保留的有效数字也将相应地存在差异. 

由于以上特性, `Decimal`也可被应用于部分科学计算领域. 用于在部分科研实验数据的处理和分析过程中, 解决与有效数字, 以及特殊修约方法有关的要求. 

In [5]:
#分别计算包含3至10位有效数字的十进制小数2.56的平方, 平方根, 考察对应结果保留的有效数字数目. 
[(Decimal(deci_descr) ** 2, Decimal(deci_descr).sqrt()) \
    for deci_descr in ["2.56" + "0" * digit for digit in range(0, 8)]]

[(Decimal('6.5536'), Decimal('1.6')),
 (Decimal('6.553600'), Decimal('1.60')),
 (Decimal('6.55360000'), Decimal('1.60')),
 (Decimal('6.5536000000'), Decimal('1.600')),
 (Decimal('6.553600000000'), Decimal('1.600')),
 (Decimal('6.55360000000000'), Decimal('1.6000')),
 (Decimal('6.5536000000000000'), Decimal('1.6000')),
 (Decimal('6.553600000000000000'), Decimal('1.60000'))]

#### `decimal`模块的`decimal.Context`类及其应用
`decimal`模块中的`decimal.Context`类的实例, 作为`decimal`模块的管理上下文, 被用于指定模块对`Decimal`对象的修约和精度管理方法. 

* 获取当前采用的管理上下文, 需调用
    ```python
    decimal.getcontext()
    ```
    方法, 两种方法均返回一个`decimal.Context`对象; 
    
* 为`decimal`模块重新指定管理上下文, 需调用
    ```python
    decimal.setcontext(cntx)
    ```
    方法. 该方法需传入一个`decimal.Context`对象`cntx`, 作为重新指定的上下文. 
    
* 修改`decimal`模块当前采用的上下文的个别属性值, 需调用
    ```python
    decimal.getcontext().prop = val
    ```
    其中, `prop`和`val`分别是待修改的属性名称和修改后的值. 
    

#### `decimal.Context`对象的管理
当`decimal`模块被首次加载至当前会话时, 其采用的管理上下文为`decimal.DefaultContext`. 

在`Python 3.7` / `Decimal 1.7`中, `decimal.Context`对象中与精度和修约管理有关的属性包括: 

|属性名称|含义|在`decimal.`<br>`DefaultContext`<br>实例中的属性值|备注|
|:-|:-:|:-|:-|
|`prec`|有效数字位数|`28`||
|`Emax`|**计算结果**的<br>最大数量级|`99999`|$0 \le Emax \le MAX\_EMAX$<br>仅影响对计算结果的处理, 不影响新构造的`Decimal`对象|
|`Emin`|**计算结果**的<br>最小数量级|`-99999`|$MIN\_EMIN \le Emin \le 0$<br>仅影响对计算结果的处理, 不影响新构造的`Decimal`对象|
|`rounding`|修约方法|`"ROUND_HALF_EVEN"`|支持的修约方法: <br>`"ROUND_CEILING"`, `"ROUND_FLOOR"`, <br>`"ROUND_UP"`, `"ROUND_DOWN"`, <br>`"ROUND_HALF_UP"`, `"ROUND_HALF_DOWN"`, <br>`"ROUND_HALF_EVEN"`,`"ROUND_05UP"`|

不同修约方法的实现规则和应用领域存在差异: 

|修约方法|正小数修约|负小数修约|备注|
|:-|:-|:-|:-|
|`"ROUND_FLOOR"`|修约至比原数值更大的最小整数|与绝对值的修约方向相同|向下取整, 相当于数学上的$\lfloor x \rfloor$函数<br>(*Gauss*函数)|
|`"ROUND_CEILING"`|修约至比原数值更小的最大整数|与绝对值的修约方向相同|向上取整, 相当于数学上的$\lceil x \rceil$函数<br>|
|`"ROUND_UP"`|修约至比原数值更大的最小整数|与绝对值的修约方向相反||
|`"ROUND_DOWN"`|修约至比原数值更小的最大整数|与绝对值的修约方向相反|截断取整, 截断小数部分, 只保留整数部分<br>相当于大部分程序语言中的`trunc`函数|
|`"ROUND_HALF_UP"`|修约至距原数值最近的整数, <br>当整数不唯一时, 取最大者|与绝对值的修约方向相反|"四舍五入"取整, 易于手工计算, <br>在大部分社会生活场合普遍使用|
|`"ROUND_HALF_DOWN"`|修约至距原数值最近的整数, <br>当整数不唯一时, 取最小者|与绝对值的修约方向相反||
|`"ROUND_HALF_EVEN"`|修约至距原数值最近的整数, <br>当整数不唯一时, 修约至距<br>原数值最近的偶数|与绝对值的修约方向相反|"四舍六入五留双"取整, 减少修约过程产生<br>的系统误差, 主要使用于科学实验数据处理|
|`"ROUND_05UP"`|如果修约后的最后一位有效<br>数字是0或5, 则向上修约; <br>否则向下修约|与绝对值的修约方向相反||


#### `Decimal`对象支持的科学计算方法

* `fma(gain, bias)` 先后将增益`gain`和偏置`bias`作用于小数
    * 将定点小数先与`gain`相乘, 随后与`bias`相加, 即 $gain \cdot x + bias$
* `sqrt` 取算术平方根 $\sqrt{x}$
* `ln` 取自然对数 $\ln x$
* `log10` 取常用对数 $\lg x$
* `exp` 取以$e$为底的指数 $e^x$

上述所有方法均支持`context = Cntx`选项. 

* 该选项指定后, 将采用`Cntx`所指的`decimal.Context`管理上下文进行修约; 
* 如未指定, 则采用`Decimal`所在的`decimal`模块当前采用的管理上下文进行修约: 
    * 除`fma`方法外, 其他科学计算的修约方法**强制指定为`"ROUND_HALF_EVEN"`("四舍六入五留双")**, 与`context = Cntx`选项所指定, 或者所在`decimal`模块当前采用的的管理上下文对象的`rounding`属性无关.         
    * `fma`方法若采用以下参数: 
        ```python
        fma(1, 0, context = decimal.Context(prec = n, rounding = mthd)
        ``` 
        可将`Decimal`对象直接按照`mthd`方法修约至**`n`位有效数字**. 

#### `Decimal`对象中由修约操作衍生的方法
* `shift(n)` 十进制小数点向右移动`n`位(当`n`为负数时, 表示向左移动). 
* `normalize()` 在不改变数字大小的前提下, 去除小数部分最后一个非零数字后面的所有0. 
    * `normalize()` 方法所得的结果, 是与原数值大小相等的所有`Decimal`对象中, 有效数字最少的. 

上述所有方法均支持`context = Cntx`选项. 

* 该选项指定后, 将采用`Cntx`所指的`decimal.Context`管理上下文进行修约; 
* 如未指定, 则采用`Decimal`所在的`decimal`模块当前采用的管理上下文进行修约. 

In [6]:
# 定义待修约的定点小数样例
deci_Sample = [Decimal("42." + tail) for tail in ["40", "40", "50", "51", "60"]]; 
deci_Sample[0] = deci_Sample[0].copy_negate(); 
deci_Sample.extend([Decimal("43.50")]); 

# 定义不同的修约方法的上下文
round_descr = ["CEILING", "FLOOR","UP", "DOWN","HALF_UP", "HALF_DOWN","HALF_EVEN","05UP"]; 
round_descr = ["ROUND_" + descr for descr in round_descr]; 
round_Cntx = [deci.Context(prec = 2, rounding = descr) for descr in round_descr]

# 基于不同的修约方法, 将定点小数强制修约至2位有效数字
deci_round_result = [[x.fma(1, 0, context = cntx) 
     for x in deci_Sample] for cntx in round_Cntx]; 

# 输出表格以便比较
deci_round_table = ["Rounding Method"]; deci_round_table.extend(deci_Sample); 
[print(str(obj), end = "\t") for obj in deci_round_table]; print(end = "\n"); # 打印表头
for (idx, row) in enumerate(deci_round_result): 
    deci_record = [round_descr[idx]]; deci_record.extend(row); # 构造记录
    [print(str(obj), end = "\t") for obj in deci_record]; print(end = "\n"); # 打印记录

Rounding Method	-42.40	42.40	42.50	42.51	42.60	43.50	
ROUND_CEILING	-42	43	43	43	43	44	
ROUND_FLOOR	-43	42	42	42	42	43	
ROUND_UP	-43	43	43	43	43	44	
ROUND_DOWN	-42	42	42	42	42	43	
ROUND_HALF_UP	-42	42	43	43	43	44	
ROUND_HALF_DOWN	-42	42	42	43	43	43	
ROUND_HALF_EVEN	-42	42	42	43	43	44	
ROUND_05UP	-42	42	42	42	42	43	
